# Use Case B - Faceted search 
***

### Revised (January 2022) to read all data from the database


## Using Eurostat themes and sub-themes to search articles from the OECD's Glossary of Statistical Terms: https://stats.oecd.org/glossary/

### Installation instructions

This is a Google Colab notebook. You must have a Google account with a Google Drive. Download it from its location in GitHub and allow the code to access your Google Drive.

Launch the notebook and put your own credentials in the pyodbc.connect() call in the chunk with title "Connect to Virtuoso database" 

### Connect Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pyodbc

In [3]:
!apt-get install virtuoso-opensource

Reading package lists... Done
Building dependency tree       
Reading state information... Done
virtuoso-opensource is already the newest version (6.1.6+repack-0ubuntu9).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11

In [4]:
import pandas as pd
import numpy as np

import ipywidgets as widgets

import gensim

import pyodbc


In [5]:
import datetime

def file_name(pre,ext):
    current_time = datetime.datetime.now() 
    return pre + '_'+ str(current_time.month)+ '_' + str(current_time.day) + \
                 '_' + str(current_time.hour)+ '_' + str(current_time.minute)  +'.'+ext
    

### The data cleansing function

In [6]:
import re
import unicodedata as ud

def clean(x, quotes=True):
    if pd.isnull(x): return x  
    x = x.strip()
    
    ## make letter-question mark-letter -> letter-quote-space-letter !!! but NOT in the lists of URLs!!!
    if quotes:
        x = re.sub(r'([A-Za-z])\?([A-Za-z])','\\1\' \\2',x) ## NEW
    
    ## make letter-question mark-space lower case letter letter-quote-space letter
    x = re.sub(r'([A-Za-z])\? ([a-z])','\\1\' \\2',x) ## NEW

    ## delete ,000 commas in numbers    
    x = re.sub(r'\b(\d+),(\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## delete  000 spaces in numbers
    x = re.sub(r'\b(\d+) (\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## remove more than one spaces
    x = re.sub(r' +', ' ',x)
    
    ## remove start and end spaces
    x = re.sub(r'^ +| +$', '',x,flags=re.MULTILINE) 
    
    ## space-comma -> comma
    x = re.sub(r' \,',',',x)
    
    ## space-dot -> dot
    x = re.sub(r' \.','.',x)
    
    # x = x.encode('latin1').decode('utf-8') ## â\x80\x99
    x = ud.normalize('NFKD',x).encode('ascii', 'ignore').decode()
    
    return x

### Create a dictionary with Eurostat's themes and sub-themes

* Include some artificial ones (theme: 'Other') to match some OECD's Glossary themes.

In [7]:

themes = {'General and regional statistics/EU policies':
          ['Non-EU countries','Regions and cities','Sustainable development goals',
          'Policy indicators'],
          'Economy and finance': 
          ['Balance of payments','Comparative price levels (PPPs)','Consumer prices',
           'Exchange rates and interest rates','Government finance','National accounts (incl. GDP)'],
          'Population and social conditions':
          ['Asylum and migration','Crime','Culture','Education and training','Health',
           'Labour market','Living conditions','Population','Social protection','Sport','Youth'],
          'Industry and services': ['Short-term business statistics','Structural business statistics',
                                    'Business registers','Globalisation in businesses','Production statistics',
                                    'Tourism'],
          'Agriculture, forestry and fisheries':['Agriculture','Fisheries','Forestry'],
          'International trade':['Goods','Services'],
          'Transport':[],
          'Environment and energy':['Energy','Environment'],
          'Science, technology and digital society':['Digital economy and society','Science and technology'],
          'Other':['Methodology','Other']}



### Connect to Virtuoso database

In [8]:
c = pyodbc.connect('DRIVER=/usr/lib/odbc/virtodbc.so;HOST=lod.csd.auth.gr:1111;UID=kimon;PWD=RkhvQYZ442e2JVXLHdtW;DATABASE=ESTAT')

In [9]:
#set encoding
c.setdecoding(pyodbc.SQL_CHAR, encoding='latin-1')
c.setencoding(encoding="latin-1")

In [10]:
cursor = c.cursor()

In [11]:
def load_table(cursor,query):
  cursor.execute(query)
  t1 = cursor.fetchall()
  df = pd.DataFrame.from_records(t1, columns=[x[0] for x in cursor.description])
  return df

### Read the table with OECD's terms and definitions
* Column 'related' has the cross-references separated by semicolons with some invalid ones (not valid URL in 'related_URL') removed.


In [12]:
#import ast

query      = """SELECT id, article_id, term, url, definition, context, theme, related, related_url, last_update
                FROM ESTAT.V1.OECD_Glossary """

OECD_df = load_table(cursor,query)

OECD_df['related'] = OECD_df['related'].apply(lambda x: re.sub(r'\?','-',x) if(pd.notnull(x)) else x)
OECD_df['related'] = OECD_df['related'].apply(lambda x: x.split(';'))
OECD_df['related_url'] = OECD_df['related_url'].apply(lambda x: x.split(','))
OECD_df.head()


,id,article_id,term,url,definition,context,theme,related,related_url,last_update
0,1,1,Abatement,https://stats.oecd.org/glossary/detail.asp?ID=1,See Pollution abatement.,,Environmental statistics,[Pollution abatement],[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, March 14, 2002"
1,2,2,Absence from work due to illness,https://stats.oecd.org/glossary/detail.asp?ID=2,Absence from work due to illness refers to the...,,Health statistics,[],[],"Thursday, November 22, 2001"
2,3,3,Activity restriction - free expectancy,https://stats.oecd.org/glossary/detail.asp?ID=3,Functional limitation-free life expectancy is ...,,Health statistics,[],[],"Wednesday, October 31, 2001"
3,4,4,Acute care,https://stats.oecd.org/glossary/detail.asp?ID=4,Acute care is one in which the principal inten...,,Health statistics,"[Acute care beds, Acute care hospital staff ra...",[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, April 25, 2013"
4,5,5,Acute care beds,https://stats.oecd.org/glossary/detail.asp?ID=5,Acute care beds are beds accommodating patient...,Acute care beds have alternatively been define...,Health statistics,"[Acute care, Long-term care beds in hospitals]",[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, April 25, 2013"


* Drop records with missing values and apply data cleansing.

In [13]:
OECD_df = OECD_df.replace('',np.nan)
print(OECD_df.isnull().sum())
OECD_df.dropna(subset=['term','definition'],inplace=True)
OECD_df.reset_index(drop=True, inplace=True)
print(OECD_df.isnull().sum())

OECD_df['term'] = OECD_df['term'].apply(clean)
OECD_df['term'] = OECD_df['term'].apply(lambda x: re.sub(r'\?','-',x))
OECD_df['definition'] = OECD_df['definition'].apply(clean)
OECD_df['context'] = OECD_df['context'].apply(clean)
OECD_df.head()

id                0
article_id        0
term              3
url               0
definition        0
context        5538
theme            35
related           0
related_url       0
last_update    1763
dtype: int64
id                0
article_id        0
term              0
url               0
definition        0
context        5536
theme            35
related           0
related_url       0
last_update    1761
dtype: int64


,id,article_id,term,url,definition,context,theme,related,related_url,last_update
0,1,1,Abatement,https://stats.oecd.org/glossary/detail.asp?ID=1,See Pollution abatement.,NaN,Environmental statistics,[Pollution abatement],[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, March 14, 2002"
1,2,2,Absence from work due to illness,https://stats.oecd.org/glossary/detail.asp?ID=2,Absence from work due to illness refers to the...,NaN,Health statistics,[],[],"Thursday, November 22, 2001"
2,3,3,Activity restriction - free expectancy,https://stats.oecd.org/glossary/detail.asp?ID=3,Functional limitation-free life expectancy is ...,NaN,Health statistics,[],[],"Wednesday, October 31, 2001"
3,4,4,Acute care,https://stats.oecd.org/glossary/detail.asp?ID=4,Acute care is one in which the principal inten...,NaN,Health statistics,"[Acute care beds, Acute care hospital staff ra...",[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, April 25, 2013"
4,5,5,Acute care beds,https://stats.oecd.org/glossary/detail.asp?ID=5,Acute care beds are beds accommodating patient...,Acute care beds have alternatively been define...,Health statistics,"[Acute care, Long-term care beds in hospitals]",[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, April 25, 2013"


### Tokenize and stem the articles terms, definitions and contexts

* Also remove stop-words.
* Create columns _term tokens_, _definition tokens_, _context tokens_.

In [14]:
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import STOPWORDS
all_stopwords_gensim = STOPWORDS

p = PorterStemmer()

def text_to_words(text):
    words = str(gensim.utils.simple_preprocess(text, deacc=True))
    words = gensim.utils.tokenize(words)
    words = [word for word in words if not word in all_stopwords_gensim]
        
    words = [p.stem(token) for token in words]  
    return ' '.join(words)        

texts=list()

for i in range(len(OECD_df)):
    OECD_df.loc[i,'term tokens']=text_to_words(OECD_df.loc[i,'term'])
    OECD_df.loc[i,'definition tokens']=text_to_words(OECD_df.loc[i,'definition'])
    if not pd.isnull(OECD_df.loc[i,'context']):        
        OECD_df.loc[i,'context tokens']=text_to_words(OECD_df.loc[i,'context'])
    else:
        OECD_df.loc[i,'context tokens']=''

OECD_df.head()

,id,article_id,term,url,definition,context,theme,related,related_url,last_update,term tokens,definition tokens,context tokens
0,1,1,Abatement,https://stats.oecd.org/glossary/detail.asp?ID=1,See Pollution abatement.,NaN,Environmental statistics,[Pollution abatement],[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, March 14, 2002",abat,pollut abat,
1,2,2,Absence from work due to illness,https://stats.oecd.org/glossary/detail.asp?ID=2,Absence from work due to illness refers to the...,NaN,Health statistics,[],[],"Thursday, November 22, 2001",absenc work ill,absenc work ill refer number work dai lost yea...,
2,3,3,Activity restriction - free expectancy,https://stats.oecd.org/glossary/detail.asp?ID=3,Functional limitation-free life expectancy is ...,NaN,Health statistics,[],[],"Wednesday, October 31, 2001",activ restrict free expect,function limit free life expect averag number ...,
3,4,4,Acute care,https://stats.oecd.org/glossary/detail.asp?ID=4,Acute care is one in which the principal inten...,NaN,Health statistics,"[Acute care beds, Acute care hospital staff ra...",[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, April 25, 2013",acut care,acut care princip intent follow manag labour o...,
4,5,5,Acute care beds,https://stats.oecd.org/glossary/detail.asp?ID=5,Acute care beds are beds accommodating patient...,Acute care beds have alternatively been define...,Health statistics,"[Acute care, Long-term care beds in hospitals]",[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, April 25, 2013",acut care bed,acut care bed bed accommod patient princip cli...,acut care bed altern defin bed accommod patien...


### Read the table with the correspondence between a) Eurostat's themes and sub-themes b) OECD's Glossary themes

* There may be more than one OECD's themes corresponding to a Eurostat's theme and sub-theme combination.

In [15]:
query =      """SELECT ESTAT_theme, ESTAT_sub_theme, OECD_themes
                FROM ESTAT.V1.Eurostat_OECD_themes """

corresp_df = load_table(cursor,query)
corresp_df['OECD_themes'] = corresp_df['OECD_themes'].apply(lambda x: x.split(';')) 
corresp_df

,ESTAT_theme,ESTAT_sub_theme,OECD_themes
0,General and regional statistics/EU policies,Non-EU countries,[]
1,General and regional statistics/EU policies,Regions and cities,[]
2,General and regional statistics/EU policies,Sustainable development goals,[]
3,General and regional statistics/EU policies,Policy indicators,[]
4,Economy and finance,Balance of payments,[Financial statistics - Balance of payments]
5,Economy and finance,Comparative price levels (PPPs),[Prices and purchasing power parities - Price ...
6,Economy and finance,Consumer prices,[Prices and purchasing power parities - Price ...
7,Economy and finance,Exchange rates and interest rates,[Financial statistics - Exchange rates]
8,Economy and finance,Government finance,[Financial statistics - Government finance and...
9,Economy and finance,National accounts (incl. GDP),"[National accounts - Input-output tables, Nati..."


### Insert Eurostat's themes - sub-themes information into OECD Glossary articles dataframe

In [16]:
OECD_df['ESTAT_theme']=pd.Series(list() for i in range(len(OECD_df)))
OECD_df['ESTAT_sub_theme']=pd.Series(list() for i in range(len(OECD_df)))
for i in range(len(OECD_df)):
    theme = OECD_df.loc[i,'theme']

    for j in range(len(corresp_df)):
        if theme in corresp_df.loc[j,'OECD_themes']:
            if corresp_df.loc[j,'ESTAT_theme'] not in OECD_df.loc[i,'ESTAT_theme']: ## avoid duplicates
                OECD_df.loc[i,'ESTAT_theme'].append(corresp_df.loc[j,'ESTAT_theme'])
            if corresp_df.loc[j,'ESTAT_sub_theme'] not in OECD_df.loc[i,'ESTAT_sub_theme']: ## avoid duplicates               
                OECD_df.loc[i,'ESTAT_sub_theme'].append(corresp_df.loc[j,'ESTAT_sub_theme'])
            

idx=OECD_df[OECD_df['ESTAT_theme'].apply(len)==0].index
OECD_df.drop(index=idx,inplace=True)
OECD_df.reset_index()
OECD_df.head()

,id,article_id,term,url,definition,context,theme,related,related_url,last_update,term tokens,definition tokens,context tokens,ESTAT_theme,ESTAT_sub_theme
0,1,1,Abatement,https://stats.oecd.org/glossary/detail.asp?ID=1,See Pollution abatement.,NaN,Environmental statistics,[Pollution abatement],[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, March 14, 2002",abat,pollut abat,,[Environment and energy],"[Energy, Environment]"
1,2,2,Absence from work due to illness,https://stats.oecd.org/glossary/detail.asp?ID=2,Absence from work due to illness refers to the...,NaN,Health statistics,[],[],"Thursday, November 22, 2001",absenc work ill,absenc work ill refer number work dai lost yea...,,[Population and social conditions],[Health]
2,3,3,Activity restriction - free expectancy,https://stats.oecd.org/glossary/detail.asp?ID=3,Functional limitation-free life expectancy is ...,NaN,Health statistics,[],[],"Wednesday, October 31, 2001",activ restrict free expect,function limit free life expect averag number ...,,[Population and social conditions],[Health]
3,4,4,Acute care,https://stats.oecd.org/glossary/detail.asp?ID=4,Acute care is one in which the principal inten...,NaN,Health statistics,"[Acute care beds, Acute care hospital staff ra...",[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, April 25, 2013",acut care,acut care princip intent follow manag labour o...,,[Population and social conditions],[Health]
4,5,5,Acute care beds,https://stats.oecd.org/glossary/detail.asp?ID=5,Acute care beds are beds accommodating patient...,Acute care beds have alternatively been define...,Health statistics,"[Acute care, Long-term care beds in hospitals]",[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, April 25, 2013",acut care bed,acut care bed bed accommod patient princip cli...,acut care bed altern defin bed accommod patien...,[Population and social conditions],[Health]


In [17]:
#def my_split(x,delim):
#    if pd.isna(x):
#        return []
#    else:
#        return x.split(delim)

OECD_df[["day", "month", "year"]] =OECD_df["last_update"].str.split(",", expand = True)
OECD_df['year'] =OECD_df["year"].astype(str)

OECD_df.loc[OECD_df['year'] == 'nan', 'year'] = np.nan 

OECD_df['year'].fillna(value="Not found", inplace=True)
OECD_df.reset_index(drop=True,inplace=True)
OECD_df.head()

,id,article_id,term,url,definition,context,theme,related,related_url,last_update,term tokens,definition tokens,context tokens,ESTAT_theme,ESTAT_sub_theme,day,month,year
0,1,1,Abatement,https://stats.oecd.org/glossary/detail.asp?ID=1,See Pollution abatement.,NaN,Environmental statistics,[Pollution abatement],[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, March 14, 2002",abat,pollut abat,,[Environment and energy],"[Energy, Environment]",Thursday,March 14,2002
1,2,2,Absence from work due to illness,https://stats.oecd.org/glossary/detail.asp?ID=2,Absence from work due to illness refers to the...,NaN,Health statistics,[],[],"Thursday, November 22, 2001",absenc work ill,absenc work ill refer number work dai lost yea...,,[Population and social conditions],[Health],Thursday,November 22,2001
2,3,3,Activity restriction - free expectancy,https://stats.oecd.org/glossary/detail.asp?ID=3,Functional limitation-free life expectancy is ...,NaN,Health statistics,[],[],"Wednesday, October 31, 2001",activ restrict free expect,function limit free life expect averag number ...,,[Population and social conditions],[Health],Wednesday,October 31,2001
3,4,4,Acute care,https://stats.oecd.org/glossary/detail.asp?ID=4,Acute care is one in which the principal inten...,NaN,Health statistics,"[Acute care beds, Acute care hospital staff ra...",[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, April 25, 2013",acut care,acut care princip intent follow manag labour o...,,[Population and social conditions],[Health],Thursday,April 25,2013
4,5,5,Acute care beds,https://stats.oecd.org/glossary/detail.asp?ID=5,Acute care beds are beds accommodating patient...,Acute care beds have alternatively been define...,Health statistics,"[Acute care, Long-term care beds in hospitals]",[https://stats.oecd.org/glossary/detail.asp?ID...,"Thursday, April 25, 2013",acut care bed,acut care bed bed accommod patient princip cli...,acut care bed altern defin bed accommod patien...,[Population and social conditions],[Health],Thursday,April 25,2013


### Temporarily in this file: produce also file for input to Power BI

In [18]:
OECD_content = OECD_df.copy()
OECD_content['ESTAT_theme'] = OECD_content['ESTAT_theme'].apply(lambda x: ';'.join(x))
OECD_content['ESTAT_sub_theme'] = OECD_content['ESTAT_sub_theme'].apply(lambda x: ';'.join(x))

OECD_content.to_excel('OECD_content.xlsx')

## Facets
***
### The function filtering the results.

In [19]:

df1 = None

def articles(Top_articles, Keywords, themes_dd, sub_themes_dd,year,related): 

    global df1
    
    Keywords = text_to_words(Keywords) 
    print('Searching for keywords: ',Keywords)
    df1=OECD_df[OECD_df['term tokens'].str.contains(Keywords,regex=False) | OECD_df['definition tokens'].str.contains(Keywords,regex=False) | OECD_df['context tokens'].str.contains(Keywords,regex=False)]

    if len(df1) == 0:
        print("No matches found")
        return
        
    if year != "All years":    
        df1 = df1[df1['year'].str.contains(year)]    
    if len(df1) == 0:
        print("No matches found")
        return        
        
    if themes_dd != "All themes":
        #df1 = df1[df1['ESTAT_theme'].str.contains(themes_dd,regex=False)]
        df1=df1[df1['ESTAT_theme'].apply(lambda x: themes_dd in x)]
        

    if sub_themes_dd is not None:
        if sub_themes_dd != "All sub-themes" and sub_themes_dd != "":    
            #df1 = df1[df1['ESTAT_sub_theme'].str.contains(sub_themes_dd,regex=False)]
            df1=df1[df1['ESTAT_sub_theme'].apply(lambda x: sub_themes_dd in x)]

    if len(df1) == 0:
        print("No matches found")
    else:
        df1.reset_index(inplace=True)
        print(df1.term.count()," articles found")
        h = ''
        for i in range(min(Top_articles,len(df1))):
            l,n = df1.loc[i,["url","term"]].values

            h += '<br/><u><a href="' + l + '" target="_blank">'+ n + '</a></u>'
            if related:
                

                if len(df1.loc[i,"related"]) > 0:
                    h += '<blockquote>Related links:'
                    for k in range(len(df1.loc[i,"related"])):
                        title = df1.loc[i,"related"][k]
                        url = df1.loc[i,"related_url"][k]
                        h += '<br><a href="' + url + '" target="_blank">'+ title + '</a>'
                        h += '</blockquote>'
                else:
                    h += ' None'
                
        display(HTML( h))
  
    

### The widgets.

In [20]:
from IPython.display import display
from ipywidgets import HTML
layout = widgets.Layout(width='500px', height='30px')

In [21]:
def query_build2(themes):
    style = {'description_width': 'initial'}
      
    
    
    themes_dd = widgets.Dropdown(
        description='Select theme:',
        options=['All themes']+sorted([k for k in themes.keys()]),
        style=style
    )    
    

    def on_change_theme(change):
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'All themes':
                sub_themes_dd.options = []
            else:    
                sub_themes_dd.options = ['All sub-themes']+themes[themes_dd.value]
                
                

    themes_dd.observe(on_change_theme)
    
    sub_themes_dd = widgets.Dropdown(
        description='Select sub-theme:',
        options= [''],
        style=style
    )    
    

 
   

    Top_articles = widgets.IntSlider(
        description='Display',
        tooltip='maximum:',
        value=50,
        min=1, 
        max = 200,
        style={'description_width': 'initial'}
    )


    Keywords = widgets.Text(
        value='',
        placeholder='Type something',
        description='Keywords:',
        disabled=False
    )
    

    year = widgets.Dropdown(
        options=['All years','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','Not found'],
        value='All years',
        description='Year:',
        disabled=False)


    
    related = widgets.Checkbox(
        value=False,
        description='Show related links',
        disabled=False,
        indent=True
     )    
    
    out = widgets.interactive_output(articles, {'Top_articles': Top_articles, 'Keywords': Keywords,'themes_dd':themes_dd,'sub_themes_dd':sub_themes_dd,'year':year,'related':related})


    left_box = widgets.VBox([themes_dd, sub_themes_dd])
    right_box = widgets.VBox([Keywords,related])
    box = widgets.HBox([left_box, right_box])
    display(box)
    
    display(year,Top_articles,out)
    
        

query_build2(themes)



Dropdown(description='Year:', options=('All years', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2…

IntSlider(value=50, description='Display', max=200, min=1, style=SliderStyle(description_width='initial'))

Output()